In [1]:
# importing libraries
import torch
from torch import nn
import pandas as pd
import numpy as np


In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"A {device} device was detected")


A cpu device was detected


In [3]:
#loading our training data
data = r"D:\ML Project\trainingData.csv"
trainingData = pd.read_csv(data)
trainingData.head(10)

,id,Gender,Age,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,28,0,> 2 Years,Yes,40454,26,217,1
1,2,Male,76,3,0,1-2 Year,No,33536,26,183,0
2,3,Male,47,28,0,> 2 Years,Yes,38294,26,27,1
3,4,Male,21,11,1,< 1 Year,No,28619,152,203,0
4,5,Female,29,41,1,< 1 Year,No,27496,152,39,0
5,6,Female,24,33,0,< 1 Year,Yes,2630,160,176,0
6,7,Male,23,11,0,< 1 Year,Yes,23367,152,249,0
7,8,Female,56,28,0,1-2 Year,Yes,32031,26,72,1
8,9,Female,24,3,1,< 1 Year,No,27619,152,28,0
9,10,Female,32,6,1,< 1 Year,No,28771,152,80,0


In [4]:
#loading our testing data
data = r"D:\ML Project\testingData.csv"
testingData = pd.read_csv(data)
testingData.head(10)

,id,Gender,Age,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,501,Male,23,21,0,< 1 Year,Yes,33870,152,258,0
1,502,Female,31,15,1,< 1 Year,No,33444,152,233,0
2,503,Female,31,15,1,< 1 Year,No,40315,152,103,0
3,504,Female,51,28,0,1-2 Year,Yes,39214,26,15,0
4,505,Female,23,37,1,< 1 Year,No,41919,152,288,0
5,506,Male,37,28,0,1-2 Year,Yes,42207,124,135,1
6,507,Female,25,28,0,< 1 Year,Yes,92716,152,247,0
7,508,Female,25,28,1,< 1 Year,No,74371,152,295,0
8,509,Female,22,28,1,< 1 Year,No,58936,152,274,0
9,510,Female,31,18,1,< 1 Year,No,44945,152,201,0


In [5]:
testingData.shape

(500, 11)

In [6]:
# data preprocessing
def data_prep(df):

    df= df.drop(columns=['id','Policy_Sales_Channel','Vintage'])

    df=pd.get_dummies(df,columns=['Gender'] ,prefix='Gender')

    df=pd.get_dummies(df,columns=['Vehicle_Damage'] ,prefix='Damage')

    df=pd.get_dummies(df,columns=['Previously_Insured'] ,prefix='prev_insured')

    df["Age"] = pd.cut(df['Age'], bins=[0, 29, 35, 50, 100])

    df['Age']= df['Age'].cat.codes

    df['Annual_Premium'] = pd.cut(df['Annual_Premium'], bins=[0, 30000, 35000,40000, 45000, 50000, np.inf])

    df['Annual_Premium']= df['Annual_Premium'].cat.codes

    df['Vehicle_Age'] =df['Vehicle_Age'].map({'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2})

    df.drop(columns=['Region_Code'],inplace= True)

    return df
# for training data
trainingData = data_prep(trainingData)

# for testing data
testingData = data_prep(testingData)

In [7]:
trainingData.head()

,Age,Vehicle_Age,Annual_Premium,Response,Gender_Female,Gender_Male,Damage_No,Damage_Yes,prev_insured_0,prev_insured_1
0,2,2,3,1,0,1,0,1,1,0
1,3,1,1,0,0,1,1,0,1,0
2,2,2,2,1,0,1,0,1,1,0
3,0,0,0,0,0,1,1,0,0,1
4,0,0,0,0,1,0,1,0,0,1


In [8]:
# Create our PyTorch tensors and move to CPU or GPU if available
# Extract the training data inputs and create a PyTorch tensor x (inputs)

inputs= ['Age','Vehicle_Age','Annual_Premium',"Gender_Female","Gender_Male","Damage_No","Damage_Yes","prev_insured_0", "prev_insured_1"]
x_train = torch.tensor(trainingData[inputs].values,dtype=torch.float, device=device)

# Extract the training dataoutputs and create a PyTorch tensor y (outputs)
outputs = ['Response']
y_train = torch.tensor(trainingData[outputs].values,dtype=torch.float, device=device)

In [9]:
# Create our PyTorch tensors and move to CPU or GPU if available
# Extract the testing data inputs and create a PyTorch tensor x (inputs)

inputs= ['Age','Vehicle_Age','Annual_Premium',"Gender_Female","Gender_Male","Damage_No","Damage_Yes","prev_insured_0", "prev_insured_1"]
x_test = torch.tensor(testingData[inputs].values,dtype=torch.float, device=device)

# Extract the training dataoutputs and create a PyTorch tensor y (outputs)
outputs = ['Response']
y_test = torch.tensor(testingData[outputs].values,dtype=torch.float, device=device)


In [10]:
# Building a neural network
# Number of inputs = 9
# Number of Hidden Units = 1
# Number of Hidden Layers = 100
# Activation Function = Relu
# Number of utputs = 1

predictor = nn.Sequential(nn.Linear(9, 100),
                         nn.ReLU(),
                         nn.Linear(100,1))
predictor.to(device)

Sequential(
  (0): Linear(in_features=9, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=1, bias=True)
)

In [11]:
# Training my Neural Network
import torch.optim as optim

# Using Binary Cross Entropy(BCE) as loss function
criterion = torch.nn.BCELoss()

# using a Stochastic Gradient descent SGD optimizer
optimizer = optim.SGD(predictor.parameters(), lr = 0.01 , momentum = 0.9)

#  Training my model using the training dataset 5 times
for epoch in range(200):
    totalLoss = 0
    for i in range(len(x_train)):
        y_pred = torch.sigmoid(predictor(x_train[i]))
        
        # measuring howgood the model predicted the output
        
        loss = criterion(y_pred, y_train[i])
        
        # tracking how well the model predicted
        totalLoss += loss.item()
        
        # updating the neural network
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # printing the loss after each iteration
    print(f"Total Loss after {epoch+1} iterations : {totalLoss}")

Total Loss after 1 iterations : 172.36357641123323
Total Loss after 2 iterations : 148.6466912131291
Total Loss after 3 iterations : 143.5382021296973
Total Loss after 4 iterations : 142.06741188946035
Total Loss after 5 iterations : 139.8206727969664
Total Loss after 6 iterations : 139.76872806248502
Total Loss after 7 iterations : 138.3432168588215
Total Loss after 8 iterations : 141.89229786992473
Total Loss after 9 iterations : 140.35288378296173
Total Loss after 10 iterations : 138.8067183411472
Total Loss after 11 iterations : 137.70072215392588
Total Loss after 12 iterations : 137.89586795376044
Total Loss after 13 iterations : 139.38191836054057
Total Loss after 14 iterations : 137.13312046038413
Total Loss after 15 iterations : 138.78294276428892
Total Loss after 16 iterations : 137.44412447813397
Total Loss after 17 iterations : 138.15398276354222
Total Loss after 18 iterations : 136.91887891040403
Total Loss after 19 iterations : 138.15798601215113
Total Loss after 20 iterat

Total Loss after 159 iterations : 131.30639094712203
Total Loss after 160 iterations : 130.11659645777985
Total Loss after 161 iterations : 132.5401729897817
Total Loss after 162 iterations : 130.70090916683955
Total Loss after 163 iterations : 131.12345724183956
Total Loss after 164 iterations : 131.91624998570802
Total Loss after 165 iterations : 131.23833564801384
Total Loss after 166 iterations : 131.94449069186658
Total Loss after 167 iterations : 130.95195038711287
Total Loss after 168 iterations : 130.27982498375084
Total Loss after 169 iterations : 133.6318563935776
Total Loss after 170 iterations : 129.77975627877248
Total Loss after 171 iterations : 131.9079651885013
Total Loss after 172 iterations : 131.1471748405882
Total Loss after 173 iterations : 130.87691493535084
Total Loss after 174 iterations : 131.47531968437556
Total Loss after 175 iterations : 131.70411347495553
Total Loss after 176 iterations : 131.7273780546641
Total Loss after 177 iterations : 130.7902577694123

In [12]:
# testing my model

accuracy = 0
for i in range(len(x_test)):
    y_pred = torch.sigmoid(predictor(x_test[i]))
    
    #applying threshold of 0.5
    if y_pred >0.5 :
        prediction = 1.0
    else:
        prediction = 0.0
    # checking if the prediction was correct    
    if prediction == y_test[i]:
        accuracy += 1
    
print(f"Accuracy of the model : {(accuracy*100)/500}%")

Accuracy of the model : 85.6%
